# Wordle

### Strategy 1
* Find the most common letters
* Find words without any repeating letters made of the most common letters

In [112]:
# Load the autoreload extension
%load_ext autoreload

# Autoreload reloads modules before executing code
# 0: disable
# 1: reload modules imported with %aimport
# 2: reload all modules, except those excluded by %aimport
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
import os
import nltk

from nltk.corpus import words

SRC_DIR = os.path.realpath(os.path.join(os.path.curdir, '..'))
NLTK_DIR = os.path.join(SRC_DIR, 'src/makewords/nltk_data')
nltk.data.path.append(NLTK_DIR)

In [230]:
# Prepare for analysis

import string
from itertools import chain

import numpy as np
import pandas as pd

zs = ['z{}'.format(n) for n in range(6)]  # for counting appearances, index 0-5
ns = ['n{}'.format(n) for n in range(1,7)]  # for counting n appearances per word

def get_clean_df():
    df = pd.DataFrame(
        0,
        index=np.arange(26),
        columns=list(chain(['total'], list(zs), list(ns)))
    )
    df.insert(
        0,
        'letters',
        list(string.ascii_lowercase)
    )
    df = df.set_index('letters')
    return df

df = get_clean_df()
df.head(4)

,total,z0,z1,z2,z3,z4,z5,n1,n2,n3,n4,n5,n6
letters,,,,,,,,,,,,,
a,0,0,0,0,0,0,0,0,0,0,0,0,0
b,0,0,0,0,0,0,0,0,0,0,0,0,0
c,0,0,0,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0,0,0,0


In [231]:
# Begin analysis
import collections

# Write this out to a file
six_letter_words = [word.lower() for word in words.words() if len(word)==6]

def get_possible_words(df, letters=None, words=None):
    for word in six_letter_words:
        if letters is not None:
            for letter in letters:
                if letter not in word:
                    continue
        for i,j in enumerate(word):
            df.loc[j]['total'] += 1          # count the word once
            df.loc[j]['z{}'.format(i)] += 1  # count each register
        letter_count = dict(collections.Counter(word))
        for letter,n in letter_count.items():
            df.loc[letter]['n{}'.format(n)] += 1  # count appearances per word
    return df

df = get_possible_words(df)
df.head(4)

,total,z0,z1,z2,z3,z4,z5,n1,n2,n3,n4,n5,n6
letters,,,,,,,,,,,,,
a,10944,1307,3282,1349,1687,1886,1433,7008,1692,184,0,0,0
b,2762,1323,117,591,507,135,89,2222,246,16,0,0,0
c,4057,1445,312,817,511,542,430,3460,288,7,0,0,0
d,3761,822,103,650,803,262,1121,3088,309,17,1,0,0


In [233]:
def first_guess(df):
    # Take the most common letters
    chars = df['total'].sort_values(axis='index', ascending=False).head(6).keys().to_list()

    # Find words containing the most common letters with no repeats
    occurrences_by_register = (
        df.sort_values(by='total', ascending=False)
          .head(6)[zs]
          .stack()
          .sort_values(axis='index', ascending=False)
          .keys()
    )

    # Build the first guess solely by sorting based on relative occurences per register
    guess = {}
    for letter, reg in occurrences_by_register:
        if reg not in guess:
            if letter not in guess.values():
                guess[reg] = letter

    import operator
    f = operator.itemgetter(0)
    out = list(guess.items())
    out.sort(key=lambda x: x[0])
    res = ''.join([l for _,l in out])
    return res

res = first_guess(df)
print(res)

oanier


In [ ]:
def next_guess(df, green, yellow):
    
green=''
yellow='oer'
